In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip /content/drive/MyDrive/Projects/Skin_disease_predictor/Skin_diseases.zip

Streaming output truncated to the last 5000 lines.
  inflating: Skin_diseases/train/Seborrheic Keratoses and other Benign Tumors/keratoacanthoma-125.jpg  
  inflating: Skin_diseases/train/Seborrheic Keratoses and other Benign Tumors/keratoacanthoma-126.jpg  
  inflating: Skin_diseases/train/Seborrheic Keratoses and other Benign Tumors/keratoacanthoma-127.jpg  
  inflating: Skin_diseases/train/Seborrheic Keratoses and other Benign Tumors/keratoacanthoma-128.jpg  
  inflating: Skin_diseases/train/Seborrheic Keratoses and other Benign Tumors/keratoacanthoma-129.jpg  
  inflating: Skin_diseases/train/Seborrheic Keratoses and other Benign Tumors/keratoacanthoma-13.jpg  
  inflating: Skin_diseases/train/Seborrheic Keratoses and other Benign Tumors/keratoacanthoma-130.jpg  
  inflating: Skin_diseases/train/Seborrheic Keratoses and other Benign Tumors/keratoacanthoma-133.jpg  
  inflating: Skin_diseases/train/Seborrheic Keratoses and other Benign Tumors/keratoacanthoma-134.jpg  
  inflating: S

In [3]:
from keras.layers import Dense, GlobalAveragePooling2D, concatenate
from keras.models import Model
from keras.applications.vgg19 import VGG19
from keras.applications.resnet import ResNet50
from keras.applications.inception_v3 import InceptionV3
from keras.applications.densenet import DenseNet121
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from tensorflow.keras.layers import Input

In [4]:
vgg19_input = Input(shape=(256, 256, 3))
resnet50_input = Input(shape=(256, 256, 3))
inceptionv3_input = Input(shape=(256, 256, 3))
densenet121_input = Input(shape=(256, 256, 3))

In [5]:
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

In [6]:
train_generator = train_datagen.flow_from_directory(
    directory="/content/Skin_diseases/train",
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical')

val_generator = val_datagen.flow_from_directory(
    directory="/content/Skin_diseases/test",
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical')

Found 15822 images belonging to 37 classes.
Found 4174 images belonging to 37 classes.


In [7]:
train,label=train_generator.next()
train.shape

(32, 256, 256, 3)

In [8]:
# load the pretrained models
vgg19_model = VGG19(weights='imagenet', include_top=False, input_tensor=vgg19_input)
resnet50_model = ResNet50(weights='imagenet', include_top=False, input_tensor=resnet50_input)
inceptionv3_model = InceptionV3(weights='imagenet', include_top=False, input_tensor=inceptionv3_input)
densenet121_model = DenseNet121(weights='imagenet', include_top=False, input_tensor=densenet121_input)

29084464/29084464 [==============================] - 2s 0us/step


In [9]:

for i, layer in enumerate(vgg19_model.layers):
    layer._name = 'vgg19_' + str(i) + '_' + layer.name


for i, layer in enumerate(resnet50_model.layers):
    layer._name = 'resnet50_model_' + str(i) + '_' + layer.name


for i, layer in enumerate(inceptionv3_model.layers):
    layer._name = 'inceptionv3_model' + str(i) + '_' + layer.name


for i, layer in enumerate(densenet121_model.layers):
    layer._name = 'densenet121_model' + str(i) + '_' + layer.name

In [10]:
# set trainable layers
for layer in vgg19_model.layers:
    layer.trainable = False
for layer in resnet50_model.layers:
    layer.trainable = False
for layer in inceptionv3_model.layers:
    layer.trainable = False
for layer in densenet121_model.layers:
    layer.trainable = False

In [11]:
vgg19_output = vgg19_model.output
resnet50_output = resnet50_model.output
inceptionv3_output = inceptionv3_model.output
densenet121_output = densenet121_model.output

In [12]:
vgg19_output = GlobalAveragePooling2D()(vgg19_output)
resnet50_output = GlobalAveragePooling2D()(resnet50_output)
inceptionv3_output = GlobalAveragePooling2D()(inceptionv3_output)
densenet121_output = GlobalAveragePooling2D()(densenet121_output)

In [13]:
concatenated = concatenate([vgg19_output, resnet50_output, inceptionv3_output, densenet121_output])

In [14]:
x = Dense(1024, activation='relu')(concatenated)
x = Dense(512, activation='relu')(x)
x = Dense(37, activation='softmax')(x)

In [15]:
model = Model(inputs=[vgg19_input, resnet50_input, inceptionv3_input, densenet121_input], outputs=x)

In [16]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 densenet121_model0_input_4 (In  [(None, 256, 256, 3  0          []                               
 putLayer)                      )]                                                                
                                                                                                  
 densenet121_model1_zero_paddin  (None, 262, 262, 3)  0          ['densenet121_model0_input_4[0][0
 g2d (ZeroPadding2D)                                             ]']                              
                                                                                                  
 densenet121_model2_conv1/conv   (None, 128, 128, 64  9408       ['densenet121_model1_zero_padding
 (Conv2D)                       )                                2d[0][0]']                   

In [17]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [18]:
# train the model
train_steps = len(train_generator)
val_steps = len(val_generator)
print(train_steps)

In [19]:
train_steps = train_generator.n // train_generator.batch_size
val_steps = val_generator.n // val_generator.batch_size
print(train_steps)

In [20]:
from keras.callbacks import ModelCheckpoint,EarlyStopping
#EarlyStopping-Stop training when a monitored metric(loss or accuracy) has stopped improving.
es=EarlyStopping(monitor='val_accuracy',min_delta=0.01,patience=3,verbose=1)
#ModelCheckpoint-to save a model or weights (in a checkpoint file) at some interval, so the model or weights can be loaded later to continue the training from the state saved.
mc=ModelCheckpoint(filepath="best_model.h5",monitor='val_accuracy',minDelta=0.01,patience=5,verbose=1,save_best_only=True)
cb=[es,mc] #it is an array

In [21]:
def multi_generator(train_gen):
    while True:
        batch,label=train_gen.next()
        ans=[batch,batch,batch,batch],label
        yield ans

In [22]:
#multi_generator(train_generator)

In [23]:
# import itertools

# def multi_generator(generators):
#     while True:
#         batches = [next(g) for g in generators]
#         yield list(itertools.chain(*batches))

In [24]:
vgg19_batch = train_generator.next()
vgg19_batch[0]

In [25]:
# def multi_generator(*generators):
#     while True:
#         for gen in generators:
#             x, y = gen.next()
#             yield x, y

In [26]:
# generator = multi_generator(train_generator, val_generator)

In [ ]:
model.fit_generator(
    generator=multi_generator(train_generator),
    steps_per_epoch=train_steps,
    epochs=10,
    validation_data=val_generator,
    validation_steps=val_steps)

In [ ]:
print(model.input_shape)

In [28]:
history = model.fit_generator(
    generator=multi_generator(train_generator),
    steps_per_epoch=train_steps,
    epochs=20,
    callbacks=cb,
    validation_data=multi_generator(val_generator),
    validation_steps=val_steps
)

<ipython-input-28-a751a7914dd1>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/20
 57/494 [==>...........................] - ETA: 4:30 - loss: 2.2355 - accuracy: 0.3536

KeyboardInterrupt: ignored

Prediction

In [ ]:
train_generator.class_indices

In [ ]:
zip(list(train_generator.class_indices.values()),list(train_generator.class_indices.keys()))

In [ ]:
diseases_names=dict(zip(list(train_generator.class_indices.values()),list(train_generator.class_indices.keys())))
diseases_names

In [ ]:
import pickle

pickle.dump(diseases_names,open('skin_diseases_names.pkl','wb'))

In [ ]:
from keras.models import load_model
model=load_model("/content/drive/MyDrive/Projects/Skin_disease_predictor/best_skin_model.h5")

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
img = load_img('/content/Skin_diseases/test/Acne and Rosacea Photos/07RosaceaMilia0120.jpg', target_size=(256, 256,3))
img_array = img_to_array(img)
print(img_array.shape)
img_array = np.expand_dims(img_array, axis=0)
print(img_array.shape)
img_array = img_array.astype('float32') / 255
print(img_array)

In [ ]:
# Making a prediction
prediction = model.predict([img_array, img_array, img_array, img_array])

In [ ]:
predicted_class = np.argmax(prediction)
predicted_label = diseases_names[predicted_class]
print(predicted_label)